# Question 2 (BLP)

We assume the demand model $$u_{ijt} = X_{jt}\beta + \sigma_{B}\nu_{i} + \alpha p_{jt} + \sigma_{I}I_{i}p_{jt} + \xi_{jt} + \epsilon_{ijt}$$ and use BLP to estimate it. The steps are detailed below:

## Step 1: Simulate draws

The logit specification of our model implies that our shares can be written as an integral of fractions of utility components. Specifically, we can adapt the formulas from class to our model: $$s_{jt} = \int \frac{X_{jt}\beta + \sigma_{B}\nu_{i} + \alpha p_{jt} + \sigma_{I}I_{it}p_{jt} + \xi_{jt}}{1+\sum_{m=1}^{J}X_{mt}\beta + \sigma_{B}\nu_{i} + \alpha p_{mt} + \sigma_{I}I_{it}p_{mt} + \xi_{mt}}dP_{I}(I)dP_{\nu}(\nu)$$ 
While this integral cannot be solved analytically, we can simulate random draws of income (uniformly over the income data we have for each product-market) and normal demand shocks to estimate shares conditional on demand parameters:
$$\hat{s}_{jt} = \sum_{r}\frac{X_{jt}\beta + \sigma_{B}\nu_{r} + \alpha p_{jt} + \sigma_{I}I_{rt}p_{jt} + \xi_{jt}}{1+\sum_{m=1}^{J}X_{mt}\beta + \sigma_{B}\nu_{r} + \alpha p_{mt} + \sigma_{I}I_{rt}p_{mt} + \xi_{mt}}$$
We code a function that returns this share while taking demand parameters as input below.

To get closer to the notation used in the lecture notes ($\beta$ is used differently in the problem set,) we will express the deterministic component of utility as $\delta_{jt}$ = $X_{jt}\beta + \alpha p_{jt} + \xi_{jt}$ and the variables that interact with the idiosyncratic components as $\sigma = (\sigma_{B},\sigma_{I})$.

In [32]:
import numpy as np
import pandas as pd
import os
import scipy

data = pd.read_csv('./cleaned_data/data.csv')
data = data.drop(columns='Unnamed: 0')

demo = pd.read_csv('./PS1_Data/OTCDemographics.csv',sep='\t')
data = pd.merge(data,demo,how='left',left_on=['store','week'],right_on=['store','week'],validate='m:1')
data = data[['store','week','brand','sales_','price_','prom_','brand_2','brand_3','brand_4','brand_5','brand_6','brand_7','brand_8','brand_9','brand_10','brand_11','hhincome1','hhincome2','hhincome3','hhincome4','hhincome5','hhincome6','hhincome7','hhincome8','hhincome9','hhincome10','hhincome11','hhincome12','hhincome13','hhincome14','hhincome15','hhincome16','hhincome17','hhincome18','hhincome19','hhincome20','count', 'ms_naught', 'ms_by_store_week']]
data.head()

,store,week,brand,sales_,price_,prom_,brand_2,brand_3,brand_4,brand_5,...,hhincome14,hhincome15,hhincome16,hhincome17,hhincome18,hhincome19,hhincome20,count,ms_naught,ms_by_store_week
0,2,1,1,16,3.29,0.0,0,0,0,0,...,11.096550,9.074136,11.56383,10.475980,11.31761,10.95628,10.73356,14181,0.994077,0.001128
1,2,2,1,12,3.27,0.0,0,0,0,0,...,10.461870,10.513160,11.65730,8.476680,11.32159,10.28266,10.02471,13965,0.994271,0.000859
2,2,3,1,6,3.37,0.0,0,0,0,0,...,8.676463,10.341440,11.64640,9.590904,11.96942,11.36072,11.35747,13538,0.995568,0.000443
3,2,4,1,12,3.30,0.0,0,0,0,0,...,11.080500,8.894314,10.27435,11.675790,12.48191,10.79339,11.42795,13735,0.994321,0.000874
4,2,5,1,10,3.34,0.0,0,0,0,0,...,11.982330,10.211610,10.33920,11.067370,11.89204,10.28755,11.68925,13735,0.995268,0.000728


In [37]:
import random
R = 100

# Simulate draws R=100 times and average out shares
def sim_shares(master,sigma_B,sigma_I, alpha):

    # Keep covariates and intermediate variables
    # master = master[['store','week','brand','sales_','price_','prom_','brand_2','brand_3','brand_4','brand_5','brand_6','brand_7','brand_8','brand_9','brand_10','brand_11','hhincome1','hhincome2','hhincome3','hhincome4','hhincome5','hhincome6','hhincome7','hhincome8','hhincome9','hhincome10','hhincome11','hhincome12','hhincome13','hhincome14','hhincome15','hhincome16','hhincome17','hhincome18','hhincome19','hhincome20','w_old','w_new','count']]
    
    # tot is total of all simulated shares
    master['tot'] = 0

    # Simulate draws
    for i in range(R):
        data_copy = master.copy()
        
        # Demand shock
        nu = np.random.normal()
        # Choose income randomly
        hh = random.randint(1,20)

        data_copy['V'] = data_copy['w_old']*np.exp(sigma_B*nu + sigma_I*data_copy[f'hhincome{hh}']*data_copy['price_'])
        # Use logit to calculate product shares in market
        data_sum = data_copy.groupby(['store', 'week'],as_index=False)['V'].sum()
        data_sum.rename(columns={'V':'sum'},inplace=True)
        data_sum['sum'] = data_sum['sum'] + 1
        data_copy = pd.merge(data_copy,data_sum,how='left',left_on=['store','week'],right_on=['store','week'],validate='m:1')
        s = data_copy['V']/data_copy['sum']
        master['tot'] += s

    # Average share
    master['est'] = master['tot']/R
    return master

## Step 2: Contraction Mapping

Given $sigma_{B}, sigma_{I}$, we can iterate the contraction mapping in the lecture notes to approximate a value of $\delta_{jt}$ for each product-market pair that results in a share close to the actual shares in the data. The equation we iterate to approximate $\delta_{jt}$ is:
$$\exp(\delta^{i+1}_{jt}) = \exp(\delta^{i}_{jt})\frac{s_{jt}^{0}}{s_{jt}(\delta^{i}_{jt},\beta)}$$

In [54]:
# Calculate delta: deterministic component of jt-level utility
def calc_delta(orig,sigma_B,sigma_I,alpha=0):
    # Initialize search values and threshold
    epsilon = 0.0001
    orig['w_old'] = np.exp(orig['ms_by_store_week'])
    orig['w_new'] = 0
    count = 0

    # Iterate contraction mapping until threshold is found
    while True:
        if count > 100:
            print(np.average((orig['ms_by_store_week'] - orig['est']).abs()))
            print(orig['est'])
            break
        if any(orig['w_old'].isnull()):
            raise Exception("NaNs")
        # orig['delta'] = np.log(orig['w_old'])
        orig = sim_shares(orig,sigma_B,sigma_I, alpha)
        orig['w_new'] = orig['w_old']*orig['ms_by_store_week']/orig['est']
        
        if np.average((orig['ms_by_store_week'] - orig['est']).abs()) < epsilon:
            break
        orig['w_old'] = orig['w_new']
        count += 1

    return np.log(orig['w_new']).to_numpy()

# Calculate xi: our jt-level residual
# Two parts: iterate contraction mapping, then subtract out linear terms given beta
# def calc_xi_(orig,sigma_B,sigma_I,beta, alpha=0):

#     # Calculate delta
#     orig['delta'] = calc_delta(orig,sigma_B,sigma_I, alpha)    

#     # Calculate xi: take linear component out of delta
#     orig['xi'] = orig['delta'] - orig[['price_','brand_2','brand_3','brand_4','brand_5','brand_6','brand_7','brand_8','brand_9','brand_10','brand_11']].dot(beta)
#     return orig['xi'].to_numpy()

def calc_xi(beta, data, delta, fields):
    return sum((delta - data[fields].dot(beta))**2)
    
def find_beta(data, delta, sigma_B, sigma_I):
    beta_fields = ['price_','brand_2','brand_3','brand_4','brand_5','brand_6','brand_7','brand_8','brand_9','brand_10','brand_11']
    beta = np.ones(11)

    return scipy.optimize.minimize(calc_xi, beta, args=(data, delta, beta_fields))

Below I try to run the above code step by step. Keeping the mess so that you can see the (lack of) convergence: the outputted numbers are the difference between successive values of $\delta_{jt}$, averaged over all jt-pairs. It looks like there's initial convergence, but eventually it starts to explode...

In [41]:
sigma_is = [-i*0.01 for i in range(10)]
sigma_bs = [-i*0.01 for i in range(10)]
# alphas = [-i*0.5 for i in range(3)]

In [42]:

master = data.copy()

deltas = [[None for _ in sigma_is] for _ in sigma_bs]
for i, ii in enumerate(sigma_is):
    for b, bb in enumerate(sigma_bs):
        # for a, aa in enumerate(alphas):
        try:
            delta = calc_delta(master,bb , ii) 
            deltas[i][b] = delta
        except Exception as e:
            print(e)

[[array([-6.78074497, -7.0532989 , -7.71683945, ..., -8.05928005,
         -7.58860252, -7.93386057]),
  array([-6.78026886, -7.05282278, -7.71636334, ..., -8.05880394,
         -7.58812641, -7.93338446]),
  array([-6.78435003, -7.05690487, -7.72045142, ..., -8.06288945,
         -7.59220358, -7.93746132]),
  array([-6.78627359, -7.05882932, -7.7223817 , ..., -8.06481721,
         -7.59412332, -7.93938075]),
  array([-6.78295276, -7.05550722, -7.71905126, ..., -8.06149038,
         -7.59080792, -7.93606578]),
  array([-6.78759811, -7.06015225, -7.72369394, ..., -8.0661341 ,
         -7.59545463, -7.94071259]),
  array([-6.78814686, -7.060704  , -7.72426567, ..., -8.06669719,
         -7.59599041, -7.94124735]),
  array([-6.7836977 , -7.05625283, -7.71980112, ..., -8.06223843,
         -7.59154992, -7.93680755]),
  array([-6.78222364, -7.05478016, -7.71833773, ..., -8.06077103,
         -7.59006988, -7.93532704]),
  array([-6.79966934, -7.07222965, -7.73581198, ..., -8.07823466,
       

In [ ]:

for i, ii in enumerate(sigma_is):
    for b, bb in enumerate(sigma_bs):
        print(f'{ii}, {bb}')
        res = find_beta(master, deltas[i][b], b, i)
        print(f"Success: {res.success}")
        print(res.x)
    


0.0, 0.0
Success: False
[-1.93853373  2.53539855  6.09778133 -1.74073843  1.97902563  6.99418461
 -3.40517337 -1.84802934 -0.2060569  -4.15115771  0.28117711]
0.0, -0.01
Success: False
[-1.93840954  2.5352606   6.09738581 -1.74063079  1.97886294  6.99364826
 -3.40502897 -1.84800143 -0.20607228 -4.15092092  0.28110204]
0.0, -0.02
Success: False
[-1.93948147  2.53647454  6.10082286 -1.74153776  1.98029445  6.99831044
 -3.40624781 -1.84821773 -0.20590465 -4.15293789  0.28178493]
0.0, -0.03
Success: False
[-1.93998664  2.53704458  6.10244081 -1.74196673  1.98096752  7.00050599
 -3.40682356 -1.84832199 -0.20582722 -4.15388935  0.28210497]
0.0, -0.04
Success: False
[-1.93911432  2.5360588   6.09964534 -1.74122722  1.97980429  6.99671307
 -3.40583102 -1.84814429 -0.20596279 -4.15224715  0.28155066]
0.0, -0.05
Success: False
[-1.94033245  2.53743428  6.1035467  -1.74226036  1.98142732  7.00200797
 -3.40721827 -1.84839513 -0.20577502 -4.15454212  0.28232405]
0.0, -0.06
Success: False
[-1.940479

## Step 3: Define GMM objective function

We now use our instruments to define an objective function which is to be minimized to find our optimal paramters $\beta$, $\sigma_{B}$, and $\sigma_{I}$. Using the formula found in Nevo's RA guide, we can express $\beta$ as a function of $(\sigma_{B},\sigma_{I})$: 
$$\beta = (X^{T}ZWZ^{T}X)^{-1}X^{T}ZWZ^{T}\delta(\sigma_{B},\sigma_{I})$$  
With $\beta$ in hand, we can now calculate $\xi(\sigma_{B},\sigma_{I},\beta)$ and thus our entire objective function:
$$\xi^{T}ZWZ^{T}\xi$$

In [158]:
instr = pd.read_csv('./PS1_Data/OTCDataInstruments.csv',sep='\t')
instr = instr.drop(columns=['store','week','brand','avoutprice'])
Z = instr.to_numpy()
W = np.linalg.inv(np.matmul(np.transpose(Z),Z))
X = orig[['price_','brand_2','brand_3','brand_4','brand_5','brand_6','brand_7','brand_8','brand_9','brand_10','brand_11']].to_numpy()

def gmm_obj(sigma):
    sigma_B = sigma[0]
    sigma_I = sigma[1]
    proj = np.linalg.inv(np.matmul(np.transpose(X),np.matmul(Z,np.matmul(W,np.matmul(np.transpose(Z),X)))))
    vect = np.matmul(np.transpose(X),np.matmul(Z,np.matmul(W,np.matmul(np.transpose(Z),calc_delta(data,sigma_B,sigma_I)))))
    beta = np.matmul(proj,vect)
    xi = calc_xi(data,sigma_B,sigma_I,beta)
    ans = np.matmul(np.transpose(xi),np.matmul(Z,np.matmul(W,np.matmul(np.transpose(Z),xi))))
    first_comp = np.array([1,0])
    second_comp = np.array([0,1])
    return [np.matmul(first_comp,ans),np.matmul(second_comp,ans)]

## Step 4: Nonlinear search over parameters

Now that we've defined a loss function to minimize, we look for parameters $\sigma_{B}, \sigma_{I}$ that minimize it. We use scipy's fsolve, which relies on MINPACK's hybrid algorithm, for nonlinear optimization.

In [ ]:
from scipy.optimize import fsolve

(sigma_B,sigma_I) = fsolve(gmm_obj,[1,1])
proj = np.linalg.inv(np.matmul(np.transpose(X),np.matmul(Z,np.matmul(W,np.matmul(np.transpose(Z),X)))))
vect = np.matmul(np.transpose(X),np.matmul(Z,np.matmul(W,np.matmul(np.transpose(Z),calc_delta(data,sigma_B,sigma_I)))))
beta = np.matmul(proj,vect)
alpha = beta[0]

## Elasticity calculation

Unlike the logit specification, elasticities under BLP need to be simulated. We will simulate: 
$$ e_{jjt} = -\frac{p_{jt}}{s_{jt}}\int (\alpha + \sigma_{I}I_{i})Pr_{ijt}(1-Pr_{ijt})dP_{D}(D)dP_{\nu}(\nu) $$ $$ e_{jkt} = \frac{p_{kt}}{s_{jt}} \int (\alpha + \sigma_{I}I_{i})Pr_{ijt}Pr_{ikt}dP_{D}(D)dP_{\nu}(\nu)$$
where $Pr_{ijt}$ is the probability of $i$ choosing $j$, simulated using the procedure in step 1.

In [ ]:
# Calculate delta for each jt-pair
data['delta'] = calc_delta(data,sigma_B,sigma_I)

In [ ]:
def calc_own_e(orig,alpha,sigma_B,sigma_I):

    # e_own is total of all simulated share-price derivatives
    orig['e_own'] = 0

    # Simulate draws
    for i in range(R):
        data_copy = master.copy()
        
        # Demand shock
        nu = np.random.normal()
        # Choose income randomly
        hh = random.randint(1,20)
        
        data_copy['V'] = data_copy.apply(calc_V,axis=1,args=(sigma_B,sigma_I,nu,hh))

        # Use logit to calculate product shares in market
        data_sum = data_copy.groupby(['store', 'week'],as_index=False)['V'].sum()
        data_sum.rename(columns={'V':'sum'},inplace=True)
        data_sum['sum'] = data_sum['sum'] + 1
        data_copy = pd.merge(data_copy,data_sum,how='left',left_on=['store','week'],right_on=['store','week'],validate='m:1')
        data_copy['s'] = data_copy['V']/data_copy['sum']
        data_copy['dsdp'] = data_copy['s']*(1-data_copy['s'])*(alpha + sigma_I*data_copy['hhincome'+str(hh)])
        data_copy = data_copy[['store','week','brand','dsdp']]

        # Add this iteration to our total
        orig = pd.merge(orig,data_copy,how='left',left_on=['store','week','brand'],right_on=['store','week','brand'],validate='1:1')
        orig['e_own'] = orig['e_own'] + orig['dsdp']
        orig = orig.drop(columns=['dsdp'])

    # Calculate elasticity
    orig['e_own'] = -1*orig['price_']*orig['e_own']/R
    orig['e_own'] = orig['e_own']*orig['sales_']/orig['count']
    return orig


def calc_cross_e(orig,alpha,sigma_I,k):

    orig['e_'+str(k)] = 0
    
    for i in range(R):
        data_copy = orig.copy()
        
        # Demand shock
        nu = np.random.normal()
        # Choose income randomly
        hh = random.randint(1,20)

        # Calculate utility
        data_copy['V'] = data_copy.apply(calc_V,axis=1,args=(sigma_B,sigma_I,nu,hh))

        # Use logit to calculate product shares in market
        data_sum = data_copy.groupby(['store', 'week'],as_index=False)['V'].sum()
        data_sum.rename(columns={'V':'sum'},inplace=True)
        data_sum['sum'] = data_sum['sum'] + 1
        data_copy = pd.merge(data_copy,data_sum,how='left',left_on=['store','week'],right_on=['store','week'],validate='m:1')
        data_copy['s'] = data_copy['V']/data_copy['sum']

        data_k = orig[orig['brand']==k]
        data_k = data_k.rename(columns={'s':'s_k'})
        data_k = data_k[['store','week','brand','s_k']]
        data_copy = pd.merge(data_copy,data_k,how='left',left_on=['store','week'],right_on=['store','week'],validate='m:1')
        
        data_copy['dsdp'] = data_copy['s']*(data_copy['s_k'])*(alpha + sigma_I*data_copy['hhincome'+str(hh)])
        data_copy = data_copy[['store','week','brand','dsdp']]

        # Add this iteration to our total
        orig = pd.merge(orig,data_copy,how='left',left_on=['store','week','brand'],right_on=['store','week','brand'],validate='1:1')
        orig['e_'+str(k)] = orig['e_'+str(k)] + orig['dsdp']
        orig = orig.drop(columns=['dsdp'])

    data_k = orig[orig['brand']==k]
    data_k = data_k.rename(columns={'price_':'price_k'})
    data_k = data_k[['store','week','price_k']]
    orig = pd.merge(orig,data_k,how='left',left_on=['store','week'],right_on=['store','week'],validate='m:1')
    orig['e_'+str(k)] = orig['e_'+str(k)]*orig['price_k']*orig['sales_']/(orig['count']*R)
    
    return orig

In [ ]:
# 2b: Elasticities for store 9, week 10
data = calc_own_e(data,alpha,sigma_B,sigma_I)
for i in range(1,12):
    data = calc_cross_e(data,alpha,sigma_I,i)

data_ans = data[((data['week'] == 10) & (data['store'] == 9))]
data_ans = data_ans[['brand','e_own','e_1','e_2','e_3','e_4','e_5','e_6','e_7','e_8','e_9','e_10','e_11']]
for i in range(1,12):
    data[data['brand']==i]['e_'+str(i)] = data['e_own']